In [1]:
from pathlib import Path
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

PATH_TO_MODELS=Path('models')
PATH_TO_DATA=Path('data')

In [14]:
df=pd.read_parquet(PATH_TO_DATA / 'data_dokdelar.pqt')
# reset index (because the embedding search will return the index)
df=df.reset_index(drop=False)
df.rename(columns={'index':'doc_id_nr'},inplace=True)
# Split doc_id_nr into doc_id and nr
df['dok_id'], df['nr'] = zip(*(df.doc_id_nr.str.split('_',expand=False)))

df.shape

(477146, 7)

In [37]:
df.columns

Index(['doc_id_nr', 'Titel', 'rm', 'doktyp', 'Departement', 'Utskott',
       'dokdelar'],
      dtype='object')

In [4]:
# load embedings from PATH_TO_MODELS using torch.load
embeddings=torch.load(PATH_TO_MODELS / 'embeddings.pt',map_location=torch.device('cpu'))
# embeddings are tensors normalized to unit length in oprder to facilitate faster similarity search using dot product
# Note that the embeddings are in pytorch tensors and not numpy arrays

In [5]:
# load language model sentence-bert 
model = SentenceTransformer('KBLab/sentence-bert-swedish-cased',device='cpu')

In [6]:
# define query and encode it using the transformer model
query="Skolverkets statistik är viktig för att elever och vårdnadshavare ska kunna göra ett informerat skolval, för att rikta statliga insatser och för att kunna utvärdera olika insatser på skolområdet. Riksrevisionen har granskat om Skolverket på ett effektivt sätt uppfyller sitt uppdrag om framtagande och användning av statistik för skolval, resursfördelning samt uppföljning och utvärdering av skolverksamheten."
query_embedding = model.encode(query, convert_to_tensor=True, normalize_embeddings=True)

In [7]:
# find closest documents to query
# Since the tensors are normalized, we can use the more efficient dot-product as score function
hits = util.semantic_search(query_embedding, embeddings, score_function=util.dot_score, top_k=10)

In [51]:
# Example: Print index doktyp and title of the first 10 hits
for hit in hits[0]:
  print(df.iloc[hit['corpus_id']]['dok_id'],'::',df.iloc[hit['corpus_id']]['doktyp'], '::', df.iloc[hit['corpus_id']]['Titel'])

hab334 :: sou :: I mål - vägar vidare för att fler unga ska nå målen med sin gymnasieutbildning 
h3b371d2 :: sou :: Barns och ungas rätt vid tvångsvård (del B)
h803141 :: prop :: Tidsbegränsad lösning för att säkerställa tillgång till skolinformation
h9b370 :: sou :: Läromedelsutredningen - böckernas betydelse och elevernas tillgång till kunskap 
h8b366 :: sou :: Samverkande krafter - för stärkt kvalitet och likvärdighet inom komvux för elever med svenska som andraspråk 
h6b341 :: sou :: Statliga skolmyndigheter - för elever och barn i en bättre skola
h4b366 :: sou :: Det stämmer!
h70339 :: prop :: Riksrevisionens rapport om grundsärskolans kunskapsuppdrag
h90339 :: prop :: Riksrevisionens rapport om Pisa-undersökningen 2018
hab353d2 :: sou :: Statens ansvar för skolan - ett besluts- och kunskapsunderlag - Volym 2


In [22]:
# An other way of return results:
# make a list of corpus_ids
corpus_ids=[hit['corpus_id'] for hit in hits[0]]
# filter df on corpus_ids
df.filter(corpus_ids, axis=0)

,doc_id_nr,Titel,rm,doktyp,Departement,Utskott,dokdelar
463134,hab334_312,I mål - vägar vidare för att fler unga ska nå ...,2022,sou,None,None,att fastställa hur många gymnasieelever som ta...
242106,h3b371d2_282,Barns och ungas rätt vid tvångsvård (del B),2015,sou,None,None,kön och medborgarskap. Statistiska centralbyrå...
44456,h803141_16,Tidsbegränsad lösning för att säkerställa till...,2020/21,prop,Utbildningsdepartementet,KU,"i landet, deras adresser, kontaktuppgifter och..."
443243,h9b370_93,Läromedelsutredningen - böckernas betydelse oc...,2021,sou,None,None,utvärdera utbildningarnas kvalitet samt främja...
405925,h8b366_128,Samverkande krafter - för stärkt kvalitet och ...,2020,sou,None,None,och redovisa dem i den nationella statistiken....
339036,h6b341_242,Statliga skolmyndigheter - för elever och barn...,2018,sou,None,None,majoritet att kommunerna har förutsättningar f...
275382,h4b366_31,Det stämmer!,2016,sou,None,None,ha tillstånd. Skolinspektionen beslutar om såd...
32760,h70339_59,Riksrevisionens rapport om grundsärskolans kun...,2019/20,prop,Utbildningsdepartementet,UbU,"K A P S U P P D R A G – S T Y R N I N G , S T ..."
96282,h90339_21,Riksrevisionens rapport om Pisa-undersökningen...,2021/22,prop,Utbildningsdepartementet,UbU,verksamheten och ska se till att den bedrivs e...
471719,hab353d2_87,Statens ansvar för skolan - ett besluts- och k...,2022,sou,None,None,fastställas av regeringen. Syftet med ramverke...
